<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/master/Tp2_7506.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tp-2 Org de datos( FIUBA)


    
      
      
      
***

***
### Importacion de librerias

In [ ]:
#IMPORT FILES FROM DRIVE INTO GOOGLE-COLAB:

#STEP-1: Import Libraries

# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
#STEP-2: Autheticate E-Mail ID

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

AttributeError: ignored

In [ ]:
#STEP-3: Get File from Drive using file-ID

#2.1 Get the file
downloaded = drive.CreateFile({'id':'1RAGDjlzJ6spO5Sq8_x3UTIvxLhKAUBEt'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv') 

downloaded1 = drive.CreateFile({'id':'17pAgG9oJRK1bAFWRKkp96__zicG6yUmy'}) # replace the id with id of file you want to access
downloaded1.GetContentFile('test.csv') 

downloaded2 = drive.CreateFile({'id':'1u8v51BT7FZggIRD-eo0dQno--0wlxIhA'}) # replace the id with id of file you want to access
downloaded2.GetContentFile('sample_submission.csv') 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download("popular")
from sklearn import model_selection
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import HashingVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import f1_score,classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
from nltk.stem import PorterStemmer
from textblob import TextBlob
import xgboost as xgb

### Archivos necesarios

In [ ]:
train=pd.read_csv(r"train.csv")
train.head()

In [ ]:
test=pd.read_csv(r"test.csv")
test.tail()

In [ ]:
sample_submission=pd.read_csv(r"sample_submission.csv")
sample_submission.head()

In [ ]:
train.info()

## Generacion de algunas variables y nuevos DF

In [ ]:
train_text=train.text
train_target=train.target
test_text=test.text

##### Datasets que con nulos rellenados como 'none'
****
* Despues probar rellenando con 'nokeyword','nolocation'

In [ ]:
train_na=train.fillna(value='none')
test_na=test.fillna(value='none')
train_na['keyword+text']=train_na.text+' '+train_na.keyword
test_na['keyword+text']=test_na.text+' '+test_na.keyword
train_na['Caracteres']=train_na.text.str.len()
test_na['Caracteres']=test_na.text.str.len()
train_na.tail()

### Clasificadores

In [ ]:
# Establecemos a la regresion logistica como clasificador
# Arbol como clasificador
cls=LogisticRegression(random_state=0,solver='liblinear')
clf = DecisionTreeClassifier(random_state=0)
gbc = GradientBoostingClassifier(random_state=0)
rfc=RandomForestClassifier(random_state=0)


# *MODELOS*
 
  
   
    
     
      

# Naive Bayes

In [ ]:
# Verifico que no haya instancias nulas o filas completas nulas
df_train=df_train.dropna(how="all")

df_test=df_test.dropna(how="all")
df_train.info()

In [ ]:
# Normalizo el dataframe los registros NaN con texto
df_train=df_train.fillna({'keyword': 'sin keyword',\
                   'location': 'sin location'})
df_test=df_test.fillna({'keyword': 'sin keyword',\
                   'location': 'sin location'})
df_train.info()

In [ ]:
# Eliminamos los tweets duplicados considerando que su cantidad no tiene impacto en el análisis.
df_train=df_train.drop_duplicates('text')

df_test=df_test.drop_duplicates('text')

df_train.info()

In [ ]:
# Verificamos la existencia de valores nulos
df_train.isnull().any()

In [ ]:
df_test.isnull().any()

### **Análisis & tokenización**

In [ ]:
# Función de tokenización
import string
punctuation = set(string.punctuation)

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [ ]:
# Creo la serie 'text_vector'
df_train['text_vector']=df_train['text'].apply(tokenize)
df_train.head(2)

In [ ]:
df_test['text_vector']=df_test['text'].apply(tokenize)
df_test.head(2)

In [ ]:
# Verifico integridad de DF
df_train.info()

In [ ]:
df_test.info()

In [ ]:
# Descargo e importo lista de stopwords para filtrar
import nltk
nltk.download('stopwords')

In [ ]:
# Importo stopwords
from nltk.corpus import stopwords
stopwords.fileids()

In [ ]:
# Asigno en la variable 'stop' la categoria de stopword por la que voy a realizar el filtro
stop=stopwords.words('english')

In [ ]:
def filtro(text_vector):
  text_vector_filtrado = []
  for palabra in text_vector:
    if palabra not in stop:
      text_vector_filtrado.append(palabra)
  return(text_vector_filtrado)

In [ ]:
# Tokens filtrados por stopwords en inglés
df_train['text_vector_filtrado']=df_train['text_vector'].apply(filtro)
df_test['text_vector_filtrado']=df_test['text_vector'].apply(filtro)
df_train['text_vector_filtrado'].head()

In [ ]:
df_test['text_vector_filtrado'].head()

In [ ]:
#  Convierto mis vectores filtrados en la columna text (los transformo en una serie)
df_test['text']=df_test['text_vector_filtrado'].apply(lambda x: ' '.join(x) )
df_train['text']=df_train['text_vector_filtrado'].apply(lambda x: ' '.join(x) )

In [ ]:
# Asigno en una nueva columna el largo del vector 'text_vector'
df_train['elem_vector']=df_train['text_vector'].str.len()
df_train.head(2)

In [ ]:
# Cuento los elementos del vector filtrado 'text_vector_filtrado'
df_train['elem_vector_filtrado']=df_train['text_vector_filtrado'].str.len()
df_train.head(2)

In [ ]:
# Verifico estructura del DF
df_train.info()

### Preproceso y categorización de la variable ***Keyword***

In [ ]:
# "Limpiamos" la serie de caracteres especiales
df_train['keyword']=df_train['keyword'].str.replace('%20','_')

# Se lo aplicamos al set de test tambien
df_test['keyword']=df_test['keyword'].str.replace('%20','_')

# Verificamos actualización
df_train['keyword'].unique()

### Generación de **diccionario**

In [ ]:
# Listo para generar un diccionario y traducir la serie 'keyword'
keyword_list=df_train['keyword'].unique().tolist()
print(keyword_list)

In [ ]:
#1 Genero lista en espaniol
keyword_list_esp=['sin_keyword','en llamas','accidente','replica','accidente_avion','ambulancia','aniquilado','aniquilacion','apocalipse','armageddon','ejercito','incendio_intencional','piromano','ataque','atacado','avalancha','combate/batalla/lucha','bioterror','bioterrorismo','resplandecer/arder','resplandeciente/ardiente','hemorragia','exploto','plaga','ventisca','sangre','sangriento','estallido','bolsa_de_cadaver','embolsado_de_cadaver','bolsas_de_cadaver','bomba','bombardeado','bombardeo','colapso_de_puente','incendio_de_edificios','edificios_en_llamas','quemado','quemando/incendiando','incendio_edificios','incendio_de_matorrales/arbustos','bajas/muertes/perdidas','baja/muerte/perdida','catastrofe','catastrofico','emergencia_quimica','caida_acantilado','colapso','colapsado','colisionar','colisiono','colision','choque','estrellado','aplastamiento','aplastado','toque_de_queda','ciclon','da¤o','peligro','muerto','muerte','muertes','escombros','diluvio','inundado','demoler','demolido','demolicion','descarrilar','descarrilado','descarrilamiento','desolado','desolacion','destruir','destruido','destruccion','detonado','detonacion','devastado','devastacion','desastre','dislacado/desplazado','sequia','ahogar','ahogado','ahogamiento/ahogandose','tormenta_polvo','terremoto','electrocutar','electrocutado','emergencia','plan_de_emergencia','servicio_de_emergencia','engullido','epicentro','evacuar','evacuado','evacuacion','explotar','exploto/detonado/estallado','explosion','testigo_ocular','hambruna','fatal','fatalidades','fatalidades','miedo','fuego','camion_bombero','primeros_en_responder','llamas','aplastado','inundacion','inundaciones','inundaciones','incendio_forestal','incendio_forestales','granizo','tormenta_granizo','da¤o','peligro','peligroso','ola_de_calor','infierno','secuestro','secuestrador','secuestro','rehen','rehenes','huracan','herido/lesionado','heridas/lesiones','herida/lesion','inundado','inundacion','deslisamiento_tierra','lava','rayo','golpe_fuerte/estampido_fuerte','asesinato_en_serie','asesino_en_serie','masacre','caos','fusion','militar','alud_de_lodo','desastre_natural','desastre_nuclear','reactor_nuclear','obliterar/destruir/eliminar','borrado/destruido/eliminado','obliteracion/destruccion/eliminacion','derrame_de_petroleo','brote','pandemonio/confusion/caos','panico','tener_panico','polic¡a','cuarentena','en_cuarentena','emergencia_radiacion','tormenta','arrasado','refugiados','rescate','rescatado','rescatadores','disturbio','disturbios','escombros','ruina','tormenta_de_arena','grito','gritando','gritos','sismico','sumidero/pozo','hundiendose','sirena','sirenas','humo','tormenta_de_nieve','tormenta','camilla','fallo_estructural','bomba_suicida','suicida','bombardeo_suicida','hundido','sobrevivir','sobrevivio','sobreviviente','terrorismo','terrorista','amenaza','trueno','tormenta_electrica','tornado','tragedia','atrapado','trauma','traumatizado','problema','tsunami','tornado/torbellino','tifon','transtorno','tormenta_violenta','volcan','zona_de_guerra','arma','armas','torbellino','incendios_forestales','incendio_forestal','tormenta_de_viento','moretoneado','moretones','naufragio','restos','naufrago']
print(keyword_list_esp)

In [ ]:
#1 Genero el diccionario eng-esp de la serie 'Keyword'
keyword_dicc={}
y=0
for x in keyword_list:
  keyword_dicc[x]=keyword_list_esp[y]
  y=y+1
  #print(keyword_dicc)

print(keyword_dicc)

In [ ]:
# Mapeo los valores de las key del diccionario keyword_dicc
df_train['keyword_esp']=df_train['keyword'].map(keyword_dicc)

df_test['keyword_esp']=df_test['keyword'].map(keyword_dicc)

df_train.head(2)



In [ ]:
df_test.head(2)

In [ ]:
# Verifico el top 20 de las categorias mas frecuentes para la serie keyword 
df_train['keyword_esp'].value_counts().sort_values(ascending=False).head(20)

In [ ]:
df_test['keyword_esp'].value_counts().sort_values(ascending=False).head(20)

### → **Scikit Learn**: Ejecución de **One Hot Encoding** para clasificar

In [ ]:
# Dummifico las categorías
df_train_dummies=pd.get_dummies(df_train.keyword)
df_train_dummies.head(2)

In [ ]:
df_test_dummies=pd.get_dummies(df_test.keyword)
df_test_dummies.head(2)

In [ ]:
# Hago un join con df_train y df_train_dummies
df_train_merged = pd.concat([df_train,df_train_dummies],axis='columns')
df_train_merged.head(2)

In [ ]:
df_test_merged = pd.concat([df_test,df_test_dummies],axis='columns')
df_test_merged.head(2)

In [ ]:
# Elimino las columnas 'keyword_esp' y 'sin keyword'
df_train_merged_final=df_train_merged.drop(['keyword_esp','sin keyword'],axis='columns')
df_train_merged_final.head(2)

In [ ]:
df_test_merged_final=df_test_merged.drop(['keyword_esp','sin keyword'],axis='columns')
df_test_merged_final.head(2)

In [ ]:
# Exporto a .csv para verificar estructura
df_train_merged_final.to_csv('set_train_columna_texto_vector_filtrado.csv')

In [ ]:
df_test_merged_final.to_csv('set_test_columna_texto_vector_filtrado.csv')


In [ ]:
print(df_train_merged_final['text_vector_filtrado'])

In [ ]:
print(df_test_merged_final['text_vector_filtrado'])

Vectorizo con el feature **CountVectorizer**

In [ ]:
# Defino los documentos/vector (para fragmentar la cadena de palabras)
documents=(df_train_merged_final['text'])
documents

In [ ]:
# Se ignorara todas las palabras que se encuentren en la lista stop_words
vectorizer=CountVectorizer(stop_words='english')
#Ajustamos el conjunto de datos del documento al objeto countvectorizer
vectors=vectorizer.fit_transform(documents)
# conseguimos la lista de palabras que han sido clasificadas
names=vectorizer.get_feature_names()
names

In [ ]:
# creamos una matriz. El valor correspondiente a la (fila, columna) sera la frecuencia de esa palabra
doc_array=vectorizer.transform(documents).toarray()
doc_array

In [ ]:
# se verifica la cantidad de elementos de la lista
len(doc_array)

### Implementación de **BoW** O **Bag of Word**

In [ ]:
# convertimos la matriz en una estructura de datos
frecuency_matrix=pd.DataFrame(data=doc_array, columns=names)
frecuency_matrix

### Set de datos de **Pruebas** y de **Entrenamiento**

In [ ]:
# Divido los conjuntos de entrenamiento y test
X_train, X_test, y_train, y_test= train_test_split(df_train_merged_final['text'],\
                                                   df_train_merged_final['target'], random_state=1)

In [ ]:
print('Cant total de filas en el set de datos: {}'.format(df_train_merged_final.shape[0]))

In [ ]:
print('Cant de filas en el set de datos de entrenamiento: {}'.format(X_train.shape[0]))

In [ ]:
print('Cant de filas en el set de datos de pruebas: {}'.format(X_test.shape[0]))

### Ejecución de BoW para procesar los datos de pruebas

In [ ]:
# Iniciamos el metodo countvectorizer otra vez
count_vector=CountVectorizer()

In [ ]:
# Ajustamos el set de entrenamiento
training_data=count_vector.fit_transform(X_train)
training_data

In [ ]:
# Ajustamos el set de pruebas
testing_data=count_vector.transform(X_test)
testing_data

### Implementación de Naives Bayes Multinomial para clasificar

In [ ]:
# Iniciamos los metodos y ajustamos los datos en el clasificador
naives_bayes=MultinomialNB()
naives_bayes.fit(training_data, y_train)
naives_bayes.fit(testing_data, y_test)

In [ ]:
# Realizamos predicción con el conjunto de pruebas
predicciones=naives_bayes.predict(testing_data)

### Evaluación del modelo

Para todas las mediciones - cuyo rango de probabilidad es entre 0 y 1 - tener una puntuación cercana al valor 1 es buen estimador del comportamiento del modelo



In [ ]:
# El accuracy informa cómo el clasificador realiza la predicción correcta de tweets
print("Accuracy: ",format(accuracy_score(y_test,predicciones)))

In [ ]:
# La precisión informa la proporción de tweets que clasificamos como verdaderos
print("Métrica de Precisión del Modelo: ",format(precision_score(y_test,predicciones)))

In [ ]:
# La confianza o recall, da la proporción que realmente era verdadero y fueron clasificados como tal.
print("Confianza o recall del Modelo: ",format(f1_score(y_test, predicciones)))

# Regresion Logistica

######   **Modelo 0**: Simple

In [ ]:
#Vectorizer, simple 
vectorizer=CountVectorizer()
vectorizer.fit(train_text)
X_train=vectorizer.transform(train_text)
X_test=vectorizer.transform(test_text)
X_train

###### *Modelo 1*: Agregando stopwords y eliminando caracteres tiles y ese tipo de cosas

In [ ]:
#Vectorizer1, agregando stopwords y distintos n_grams
stop_words=stopwords.words('english')
vectorizer1=CountVectorizer(strip_accents='ascii',stop_words=stop_words,ngram_range=(1,5),max_df=0.95)
vectorizer1.fit(train_text)
X_train1=vectorizer1.transform(train_text)
X_test1=vectorizer1.transform(test_text)
X_train1

###### Modelo 2: Uso de TF-IDF 

In [ ]:
#Vectorizer2, usando TF-IDF
vectorizer2=TfidfVectorizer()
vectorizer2.fit(train_text)
X_train2=vectorizer2.transform(train_text)
X_test2=vectorizer2.transform(test_text)
X_train2

###### Modelo 3: Uso de TF-IDF,sacando acentos y analizando varios n_grams

In [ ]:
# Vectorizer3, usando TF-iDF, sacando acentos y agreando n_grams
max_n3=15
vectorizer3=TfidfVectorizer(strip_accents='ascii',ngram_range=(1,max_n3),)
vectorizer3.fit(train_text)
X_train3=vectorizer3.transform(train_text)
X_test3=vectorizer3.transform(test_text)
X_train3

###### Modelo 4: Inclusion de Lemmatizador, NO incluye uso de TF-IDF

In [ ]:
# Vectorizer4
# Creamos un lemmatizador 
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

vectorizer4=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer4.fit(train_text)
X_train4=vectorizer4.transform(train_text)
X_test4=vectorizer4.transform(test_text)
X_train4

###### Modelo 5( mejores resultados):  
- Uso de columna keyword + text
- Lemmatizador

In [ ]:
train_text_y_keyword=train_na['keyword+text']
test_text_y_keyword=test_na['keyword+text']

In [ ]:
# Utilizo keyword en mi analisis
vectorizer5=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer5.fit(train_text_y_keyword)
X_train5=vectorizer5.transform(train_text_y_keyword)
X_test5=vectorizer5.transform(test_text_y_keyword)
X_train5

###### Modelo 6: Utilizo keyword en mi analisis y TF-IDF

In [ ]:
# Utilizo keyword en mi analisis y TF-IDF
vectorizer6=TfidfVectorizer(tokenizer=LemmaTokenizer())
vectorizer6.fit(train_text_y_keyword)
X_train6=vectorizer6.transform(train_text_y_keyword)
X_test6=vectorizer6.transform(test_text_y_keyword)
X_train6

###### Modelo 7: Inclusion de columna location

In [ ]:
# Agregamos columna location 
train_text_keyword_location=train_na.text+''+train_na.keyword+''+train_na.location
test_text_keyword_location=test_na.text+''+test_na.keyword+''+test_na.location

In [ ]:
vectorizer7=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer7.fit(train_text_keyword_location)
X_train7=vectorizer7.transform(train_text_keyword_location)
X_test7=vectorizer7.transform(test_text_keyword_location)
X_train7

###### Modelo 8: Utilizamos funcion train_test_split.....
- No tiene ningun tipo de mejora al modelo, es solo para probar, se puede utlizar para calcular el Score en lugar de hacer sub
- Despues se generaliza para todo el set de entrenamiento y se predice set de test

In [ ]:
X=train_na['keyword+text']
y=train_na['target']
X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.20, random_state=0)

In [ ]:
vectorizer8=CountVectorizer(strip_accents='ascii',tokenizer=LemmaTokenizer())
vectorizer8.fit(X_train_)
X_train8=vectorizer8.transform(X_train_)
X_test8=vectorizer8.transform(X_test_)
X_train8

In [ ]:
## Calculamos Score con este metodo, sin hacer cross validation
cls.fit(X_train8,y_train_)
prediction0=cls.predict(X_test8)
print(f1_score(y_test_,prediction0))

In [ ]:
RC=RidgeClassifier()
RC.fit(X_train8,y_train_)
RC_predict=RC.predict(X_test8)
f1_score(RC_predict,y_test_)

In [ ]:
### Progabmos el score con algunos de los clasificadores
gbc.fit(X_train8,y_train_)
cls.fit(X_train8,y_train_)
prediction0=cls.predict(X_test8)
prediction1=gbc.predict(X_test8)
print('RL: {}'.format(f1_score(y_test_,prediction0)))
print('GBC: {}'.format(f1_score(y_test_,prediction1)))

## Clasificador propio
- Lemmatizador
- Cuenta cantidad de caracteres

In [ ]:
class Lemmatizer(BaseEstimator):
    def __init__(self):
        self.l = WordNetLemmatizer()
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = map(lambda r:  ' '.join([self.l.lemmatize(i.lower()) for i in r.split()]), x)
        x = np.array(list(x))
        return x

In [ ]:
def length_text(x):
    return scaler.fit_transform(np.array(x.str.len()).reshape(-1,1))

In [ ]:
lm=Lemmatizer()
vec=CountVectorizer()
lr=LogisticRegression(random_state=0,solver='liblinear')
clasificador=Pipeline([
    ('features',FeatureUnion([
        ('text',Pipeline([
            ('lm',lm),
            ('vec',vec)])),
        ('lenght',Pipeline([
            ('car',FunctionTransformer(length_text,validate=False))
        ]))
    ])),
    ('lr',lr)
])


- *Cross validation del nuevo clasificador*
- *El texto es text+keyword*

In [ ]:
cross_val_score(estimator=clasificador,X=train_text_y_keyword,y=train_na.target,cv=6,scoring='f1')

* Calculamos f1 para el nuevo clasificador  (Sigue usando el df spliteado)

In [ ]:
clasificador.fit(X_train_,y_train_)
clas_predict=clasificador.predict(X_test_)
f1_score(clas_predict,y_test_)

In [ ]:
# Ahora hacemos que utilice todo el dataset, asi tiene mayor corpus
clasificador.fit(train_text_y_keyword,train_na.target)
sample_submission['target']=clasificador.predict(test_text_y_keyword)
sample_submission.to_csv('sub_9(class).csv',index=False)

* Generalizamos para todo el set de train, asi tiene mayor corpus
* Hacemos submit con nuevo clasificador
* (No tiene el mejor resultado en kaggle, seria bueno hacer un grid search)

**Modelo con DF nulos rellenados con otros nombres**

In [ ]:

scaler = StandardScaler()
train_fill=train.fillna(value={'keyword':'nokeyword','location':'nolocation'})
test_fill=test.fillna(value={'keyword':'nokeyword','location':'nolocation'})
train_fill['text+keyword']=train_fill.text+' '+train_fill.keyword
test_fill['text+keyword']=test_fill.text+' '+test_fill.keyword
train_fill['Caracteres']=scaler.fit_transform(np.array(train_fill.text.str.len()).reshape(-1,1))
test_fill['Caracteres']=scaler.transform(np.array(test_fill.text.str.len()).reshape(-1,1))
train_fill.tail()

###  Generacion de nuevo DF para el train que incluye la cantidad de caracteres como variable.


In [ ]:
vectorizer9=TfidfVectorizer(tokenizer=LemmaTokenizer())
X_train9=vectorizer9.fit_transform(train_fill['text+keyword'])
X_test9=vectorizer9.transform(test_fill['text+keyword'])
X_train9


In [ ]:
XX_train=pd.concat([train_fill['Caracteres'].to_frame(),pd.DataFrame(X_train9.toarray())],axis=1)
YY_train=train_fill.target
XX_test=pd.concat([test_fill['Caracteres'].to_frame(),pd.DataFrame(X_test9.toarray())],axis=1)

In [ ]:
cls.fit(XX_train,YY_train)
predict=cls.predict(XX_test)
sample_submission['target']=predict
from google.colab import files
sample_submission.to_csv("sub_10.csv",index=False)
files.download('sub_10.csv')



In [ ]:

XX_train_, XX_test_, YY_train_, YY_test_ = train_test_split(XX_train, YY_train, test_size=0.20, random_state=0)
params={'C':np.logspace(-3,3,7),'solver':['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']}
cls=LogisticRegression()
grid=GridSearchCV(cls,params,cv=8,scoring='f1')
grid.fit(XX_train_,YY_train_)
print('parametros tuneados con grid search: {}'.format(grid.best_params_))
print('f1: ',grid.best_score_)

In [ ]:
grid.best_params_

# Redes 

In [ ]:
## Hacemos el test_train metodo
XX_train_, XX_test_, YY_train_, YY_test_ = train_test_split(XX_train, YY_train, test_size=0.20, random_state=0)

In [ ]:
dim=XX_train_.shape[1]
modelito=Sequential()
modelito.add(layers.Dense(5,input_dim=dim,activation='relu'))
modelito.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
modelito.compile(loss='binary_crossentropy',\
                 optimizer='adam',\
                 metrics=['accuracy'])
modelito.summary()

In [ ]:
tweets=modelito.fit(XX_train_,YY_train_,\
                    epochs=100,verbose=False,\
                    validation_data=(XX_test_,YY_test_),batch_size=50)

In [ ]:
modelito.evaluate(XX_test_,YY_test_,verbose=False)

In [ ]:
pred=modelito.predict(XX_test_)
pred_bool=np.round(pred).astype(int)
print(classification_report(YY_test_,pred_bool))

In [ ]:
def plot_history(history):

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(121)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(122)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    from keras import backend as K
### Definimos metricas para generar f1
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
plot_history(tweets)

In [ ]:
###### Generalizacion
dim1=XX_train.shape[1]
red1=Sequential()
red1.add(layers.Dense(5,input_dim=dim,activation='relu'))
red1.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
red1.compile(loss='binary_crossentropy',\
                 optimizer='adam',\
                 metrics=['accuracy'])
red1.summary()

In [ ]:
tweets1=red1.fit(XX_train,YY_train,\
                    epochs=100,verbose=False,\
                    batch_size=50)

In [ ]:
pred_red1=red1.predict(XX_test)
pred_red1=np.round(pred_red1).astype(int)
sample_submission['target']=pred_red1
sample_submission.to_csv('sub11(RN).csv',index=False)
from google.colab import files
files.download('sub11(RN).csv')



## Embeddings

DF para hacer Embedding

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train_fill['text+keyword'],train_fill['target'], test_size=0.20, random_state=0)

In [ ]:
token=Tokenizer(num_words=5000)
token.fit_on_texts(X_train)


In [ ]:
X_emb_train=token.texts_to_sequences(X_train)
X_emb_test=token.texts_to_sequences(X_test)
vocab_size=len(token.word_index)+1
maxlen=100


In [ ]:
X_emb_train=pad_sequences(X_emb_train,padding='post',maxlen=maxlen)
X_emb_test=pad_sequences(X_emb_test,padding='post',maxlen=maxlen)

In [ ]:
print(X_emb_train[0,:].shape)

In [ ]:
model2=Sequential()

In [ ]:
wdims=50
model2.add(layers.Embedding(input_dim=vocab_size,input_length=100,output_dim=dims))
model2.add(layers.GlobalMaxPool1D())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model2.summary()

In [ ]:
tweets_emb=model2.fit(X_emb_train,y_train,epochs=20,verbose=2,validation_data=(X_emb_test,y_test),\
                     batch_size=10)

In [ ]:
graf_10210=plot_history(tweets_emb)   #### 10 capas densas, 2 epochs y 10 de batch_size

In [ ]:
graf_52032=plot_history(tweets_emb)  ###5 capas densas, 20 epochs y 32 de batch_size

In [ ]:
pred=model.predict(X_emb_test)
pred_bool=np.round(pred).astype(int)
print(classification_report(y_test,pred_bool))

Generalizacion

In [ ]:
token1=Tokenizer(num_words=5000)
token1.fit_on_texts(train_fill['text+keyword'])


In [ ]:
X_emb_train=token1.texts_to_sequences(train_fill['text+keyword'])
X_emb_test=token1.texts_to_sequences(test_fill['text+keyword'])
vocab_size=len(token1.word_index)+1

In [ ]:
X_emb_train=pad_sequences(X_emb_train,padding='post',maxlen=100)
X_emb_test=pad_sequences(X_emb_test,padding='post',maxlen=100)

In [ ]:
sub12=Sequential()
dims=50
sub12.add(layers.Embedding(input_dim=vocab_size,input_length=100,output_dim=dims))
sub12.add(layers.GlobalMaxPool1D())
sub12.add(layers.Dense(10, activation='relu'))
sub12.add(layers.Dense(1, activation='sigmoid'))
sub12.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
sub12.summary()

In [ ]:
tweets_emb=sub12.fit(X_emb_train,train_fill['target'],epochs=2,verbose=2,\
                     batch_size=10)

In [ ]:
from google.colab import files
pred_emb=sub12.predict(X_emb_test)
pred_emb=np.round(pred_emb).astype(int)
sample_submission['target']=pred_emb
sample_submission.to_csv('sub12(EMB).csv',index=False)
files.download('sub12(EMB).csv')

**Embedding pre-entrenado**






In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train_fill['text+keyword'],train_fill['target'], test_size=0.20, random_state=0)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
embedding_dim = 100
embedding_matrix = create_embedding_matrix(r'glove.6B.100d.txt',token1.word_index, embedding_dim)

In [ ]:
vocab_size=len(token1.word_index)+1
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

In [ ]:
model_gl= Sequential()
model_gl.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=embedding_dim, 
                           trainable=True))
model_gl.add(layers.GlobalMaxPool1D())
model_gl.add(layers.Dense(10, activation='relu'))
model_gl.add(layers.Dense(1, activation='sigmoid'))
model_gl.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_gl.summary()


In [ ]:
tweets_gl=model_gl.fit(X_emb_train,y_train,epochs=5,verbose=2,validation_data=(X_emb_test,y_test),\
                     batch_size=10)


In [ ]:
plot_history(tweets_gl)

Generalizacion

In [ ]:
token1=Tokenizer(num_words=5000)
token1.fit_on_texts(train_fill['text+keyword'])

In [ ]:
X_emb_train_1=token1.texts_to_sequences(train_fill['text+keyword'])
X_emb_test_1=token1.texts_to_sequences(test_fill['text+keyword'])
vocab_size=len(token1.word_index)+1

In [ ]:
X_emb_train_1=pad_sequences(X_emb_train_1,padding='post',maxlen=100)
X_emb_test_1=pad_sequences(X_emb_test_1,padding='post',maxlen=100)

In [ ]:
embedding_dim = 100
embedding_matrix_1 = create_embedding_matrix(r'glove.6B.100d.txt',token1.word_index, embedding_dim)

In [ ]:
vocab_size=len(token1.word_index)+1
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

In [ ]:
sub13= Sequential()
sub13.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=50, 
                           trainable=True))
sub13.add(layers.GlobalMaxPool1D())
sub13.add(layers.Dense(10, activation='relu'))
sub13.add(layers.Dense(1, activation='sigmoid'))
sub13.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
sub13.summary()


In [ ]:
tweets_gl=sub13.fit(X_emb_train,train_fill['target'],epochs=5,verbose=2,batch_size=10)


In [ ]:
from google.colab import files
pred_gl=sub13.predict(X_emb_test)
pred_gl=np.round(pred_gl).astype(int)
sample_submission['target']=pred_gl
sample_submission.to_csv('sub13(GLOVE).csv',index=False)
files.download('sub13(GLOVE).csv')

## CONV1d

In [ ]:
emb_dim=100
conv=Sequential()
conv.add(layers.Embedding(vocab_size,emb_dim,input_length=100,weights=[embedding_matrix], trainable=True))
conv.add(layers.Conv1D(128, 5, activation='relu'))
conv.add(layers.GlobalMaxPooling1D())
conv.add(layers.Dense(10, activation='relu'))
conv.add(layers.Dense(1, activation='sigmoid'))
conv.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
conv.summary()

In [ ]:
tw_conv=conv.fit(X_emb_train,y_train,epochs=10,verbose=2,
                 validation_data=(X_emb_test,y_test),batch_size=5)

In [ ]:
plot_history(tw_conv)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
def create_model(init_mode='he_uniform'):

    model=Sequential()
    model.add(layers.Embedding(vocab_size,emb_dim,input_length=100,weights=[embedding_matrix], trainable=False))
    model.add(layers.Conv1D(num_filters,kernel_size, activation='relu',kernel_initializer=init_mode))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu',kernel_initializer=init_mode))
    model.add(layers.Dense(1, activation='sigmoid',kernel_initializer=init_mode))
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
%%time
seed = 0
np.random.seed(seed)
batch_size = 64
epochs = 10

model_CV = KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 
             'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_emb_train, y_train)

In [ ]:
print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
def create_model_1(num_filters,kernel_size,vocab_size,embedding_dim,maxlen,init_mode='he_uniform'):

    model=Sequential()
    model.add(layers.Embedding(vocab_size,emb_dim,input_length=maxlen,weights=[embedding_matrix], trainable=True))
    model.add(layers.Conv1D(num_filters,kernel_size, activation='relu',kernel_initializer=init_mode))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu',kernel_initializer=init_mode))
    model.add(layers.Dense(1, activation='sigmoid',kernel_initializer=init_mode))
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
# Tuneo de hiperparametros
%%time
seed = 0
np.random.seed(seed)
param_grid_1= dict(num_filters=[32, 64, 128],kernel_size=[3, 5, 7],
                  vocab_size=[vocab_size],embedding_dim=[embedding_dim],
                  maxlen=[maxlen],epochs=[10,20],
                  batch_size=[10,32,64])
model_1 = KerasClassifier(build_fn=create_model_1, verbose=2)
grid_1= RandomizedSearchCV(estimator=model_1, param_distributions=param_grid_1,
                           cv=4, verbose=2, n_iter=5,n_jobs=-1,scoring='f1')
grid_result_1 = grid_1.fit(X_emb_train, y_train)

In [ ]:
parametros=grid_result_1.best_params_
parametros

**Generalizacion**

In [ ]:
init_mode='he_uniform'
sub_14=Sequential()
sub_14.add(layers.Embedding(vocab_size,emb_dim,input_length=100,weights=[embedding_matrix_1], trainable=True))
sub_14.add(layers.Conv1D(128,kernel_size=7, activation='relu',kernel_initializer=init_mode))
sub_14.add(layers.GlobalMaxPooling1D())
sub_14.add(layers.Dense(10, activation='relu',kernel_initializer=init_mode))
sub_14.add(layers.Dense(1, activation='sigmoid',kernel_initializer=init_mode))
sub_14.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
sub_14.summary()

In [ ]:
%%time
tw_sub14=sub_14.fit(X_emb_train_1,train_fill['target'],epochs=20,verbose=2,batch_size=64)

In [ ]:
from google.colab import files
pred_sub_14=sub_14.predict(X_emb_test_1)
pred_sub_14=np.round(pred_sub_14).astype(int)
sample_submission['target']=pred_sub_14
sample_submission.to_csv('sub14(GLOVE_tun).csv',index=False)
files.download('sub14(GLOVE_tun).csv')

## Trabajo con dataset "limpio"

In [ ]:
cleanr = re.compile('https?://\S+|www\.\S+')
p=[]
for i in range(len(train_fill)):
    p.append(re.sub(cleanr, '', train_fill['text+keyword'].loc[i]))
train_fill['t&k_sin_url']=p
p1=[]
for i in range(len(test_fill)):
    p1.append(re.sub(cleanr, '', test_fill['text+keyword'].loc[i]))
test_fill['t&k_sin_url']=p1



In [ ]:
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()
st = PorterStemmer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text
train_fill['t&k_sin_url']=train_fill['t&k_sin_url'].apply(lambda x: clean_text(x))
test_fill['t&k_sin_url']=test_fill['t&k_sin_url'].apply(lambda x: clean_text(x))

In [ ]:
X=train_fill['t&k_sin_url']
y=np.asarray(train_fill.target)


In [ ]:
a=[]
x=train_fill['t&k_sin_url']
for i in range(len(x)):
    a.append(TextBlob(train_fill.loc[i,'t&k_sin_url']))


In [ ]:
token2=Tokenizer(num_words=5000)
token2.fit_on_texts(train_fill['t&k_sin_url'])

In [ ]:
X_emb_train_2=token2.texts_to_sequences(train_fill['t&k_sin_url'])
X_emb_test_2=token2.texts_to_sequences(test_fill['t&k_sin_url'])
vocab_size_2=len(token2.word_index)+1

In [ ]:
X_emb_train_2=pad_sequences(X_emb_train_2,padding='post',maxlen=100)
X_emb_test_2=pad_sequences(X_emb_test_2,padding='post',maxlen=100)

In [ ]:
embedding_dim = 100
embedding_matrix_2 = create_embedding_matrix(r'glove.6B.100d.txt',token2.word_index, embedding_dim)

In [ ]:
vocab_size_2=len(token2.word_index)+1
nonzero_elements_2 = np.count_nonzero(np.count_nonzero(embedding_matrix_2, axis=1))
nonzero_elements_2/ vocab_size_2

In [ ]:
init_mode='he_uniform'
sub_15=Sequential()
sub_15.add(layers.Embedding(vocab_size_2,embedding_dim,input_length=100,weights=[embedding_matrix_2], trainable=False))
sub_15.add(layers.Conv1D(128,kernel_size=7, activation='relu',kernel_initializer=init_mode))
sub_15.add(layers.GlobalMaxPooling1D())
sub_15.add(layers.Dense(10, activation='relu',kernel_initializer=init_mode))
sub_15.add(layers.Dense(1, activation='sigmoid',kernel_initializer=init_mode))
sub_15.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
sub_15.summary()

In [ ]:
%%time
tw_sub15=sub_15.fit(X_emb_train_2,train_fill['target'],epochs=20,verbose=2,batch_size=64)

In [ ]:
from google.colab import files
pred_sub_15=sub_15.predict(X_emb_test_2)
pred_sub_15=np.round(pred_sub_15).astype(int)
sample_submission['target']=pred_sub_15
sample_submission.to_csv('sub15(GLOVE_tun(t=false)).csv',index=False)
files.download('sub15(GLOVE_tun(t=false)).csv')

Mismo modelo pero con trainable=true

In [ ]:
init_mode='he_uniform'
sub_16=Sequential()
sub_16.add(layers.Embedding(vocab_size_2,embedding_dim,input_length=100,weights=[embedding_matrix_2], trainable=True))
sub_16.add(layers.Conv1D(128,kernel_size=7, activation='relu',kernel_initializer=init_mode))
sub_16.add(layers.GlobalMaxPooling1D())
sub_16.add(layers.Dense(10, activation='relu',kernel_initializer=init_mode))
sub_16.add(layers.Dense(1, activation='sigmoid',kernel_initializer=init_mode))
sub_16.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
sub_16.summary()

In [ ]:
%%time
tw_sub16=sub_16.fit(X_emb_train_2,train_fill['target'],epochs=20,verbose=2,batch_size=64)

In [ ]:
from google.colab import files
pred_sub_16=sub_16.predict(X_emb_test_2)
pred_sub_16=np.round(pred_sub_16).astype(int)
sample_submission['target']=pred_sub_16
sample_submission.to_csv('sub16(GLOVE_tun(t=true)).csv',index=False)
files.download('sub16(GLOVE_tun(t=true)).csv')

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train_fill['t&k_sin_url'],train_fill['target'], test_size=0.20, random_state=0)

In [ ]:
token3=Tokenizer(num_words=10000)
token3.fit_on_texts(X)

In [ ]:
X_emb_train_3=token3.texts_to_sequences(X)
X_emb_test_3=token3.texts_to_sequences(test_fill['t&k_sin_url'])
vocab_size_3=len(token3.word_index)+1


In [ ]:
maxlen=32
X_emb_train_3=pad_sequences(X_emb_train_3,padding='post',maxlen=maxlen)
X_emb_test_3=pad_sequences(X_emb_test_3,padding='post',maxlen=maxlen)
indices = np.arange(X_emb_train_3.shape[0])
np.random.shuffle(indices)
X_emb_train_3= X_emb_train_3[indices]
y=y[indices]

In [ ]:
X_emb_test_3[700]

In [ ]:
init_mode='he_uniform'
dims=50
modelito=Sequential()
modelito.add(layers.Embedding(input_dim=vocab_size_3,input_length=maxlen,output_dim=dims))
modelito.add(layers.Conv1D(128,kernel_size=7, activation='relu',kernel_initializer=init_mode))
modelito.add(layers.GlobalMaxPooling1D())
modelito.add(layers.Dense(32,activation='relu',kernel_initializer=init_mode))
modelito.add(layers.Dense(1, activation='sigmoid',kernel_initializer=init_mode))
modelito.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
modelito.summary()

In [ ]:
%%time
tw_sub_modelito=modelito.fit(X_emb_train_3,y,epochs=10,verbose=2,batch_size=10,validation_split=0.2)

In [ ]:
embedding_dim = 100
embedding_matrix_3 = create_embedding_matrix(r'glove.6B.100d.txt',token3.word_index, embedding_dim)

In [ ]:
vocab_size_3=len(token3.word_index)+1
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix_3, axis=1))
nonzero_elements / vocab_size_3

In [ ]:
from keras.optimizers import RMSprop
model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size_3,output_dim=25, input_length=maxlen))
model.add(layers.Conv1D(32,kernel_size=5, activation='relu'))
model.add(layers.MaxPooling1D())
model.add(layers.Conv1D(32, kernel_size=5,activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()


In [ ]:
model.compile(optimizer=RMSprop(lr=1e-4),loss='binary_crossentropy',metrics=['acc',f1_m])
history = model.fit(X_emb_train_3,y,epochs=50,batch_size=10,validation_split=0.2,verbose=2)

Modelo mas basico, usando **BOW** 

In [ ]:
pipe=Pipeline([('vec',CountVectorizer()),
               ('cls',LogisticRegression())])
modelo=pipe.fit(X,y)

In [ ]:
prediccion=model_selection.cross_val_score(modelo,X,y,cv=6,scoring='f1')
prediccion

In [ ]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', DecisionTreeClassifier(criterion= 'entropy', max_depth = 20, splitter='best', random_state=0))])
modelo= pipe.fit(X, y)

In [ ]:
prediccion=model_selection.cross_val_score(modelo,X,y,cv=6,scoring='f1')
prediccion

In [ ]:
vectorizer5=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer5.fit(X)
X_train5=vectorizer5.transform(X)
X_test5=vectorizer5.transform(test_fill['t&k_sin_url'])
X_train5

In [ ]:
model=xgb.XGBClassifier(random_state=1)
model.fit(X_train5,y)
model_selection.cross_val_score(model,X_train5,y,scoring='f1')

In [ ]:
from google.colab import files
sample_submission['target']=pred
sample_submission.to_csv('sub17(BOW_limpio).csv',index=False)
files.download('sub17(BOW_limpio).csv')
